# Summation of primes

## Problem 10

The sum of the primes below 10 is 2 + 3 + 5 + 7 = 17.

Find the sum of all the primes below two million.

## Approach

First just try brute force approach. Use basic primality tester on all x < 2e6 and sum results.

In [1]:
import time
import math
import numpy as np

In [2]:
def isprime(x):
    """return True if number is prime"""
    x = int(x)
    i = 2
    ans = True
    while i <= math.sqrt(x): # test divisibility by 2,3,...,sqrt(x)
        if x % i == 0:
            ans = False
            break
        i = i+1
    return ans

In [3]:
def primes(n):
    """return list of primes below n"""
    ps = []
    for i in range(2,n):
        if isprime(i):
            ps.append(i)
    return ps

In [4]:
t1 = time.time() # tic

ps = np.array(primes(int(2e6)))
s = np.sum(ps)

t2 = time.time() # toc
print ''
print 'result:', s
print 'seconds:', t2-t1


result: 142913828922
seconds: 30.2820339203


## Speed up

Wasting a lot of time testing numbers we know aren't prime. Starting with 2,3,..., we know that any multiple of these numbers is no longer a candidate as a prime. So instead keep track of multiples as we go, and avoid testing those numbers. 

In [5]:
def maxmultarray(x,y):
    """find maximum multiple n satisfying x*n<y, return array from x*1 to x*n"""
    n = math.floor((y-1)/x)
    na = x*(np.arange(n)+1)
    return na

In [6]:
def primes2(n):
    """return list of primes below n. skip testing multiples of numbers"""
    ps = []
    test = np.arange(2,n) # list of all numbers to test
    while len(test)>0:
        i = test[0]
        if isprime(i):
            ps.append(i)
        mults = maxmultarray(i,n) # multiples of this trial number up to n
        test = np.setdiff1d(test,mults,assume_unique=True) # find remaining numbers to test
    return ps

In [7]:
t1 = time.time() # tic

ps = np.array(primes2(int(1e5)))
s = np.sum(ps)

t2 = time.time() # toc
print ''
print 'result:', s
print 'seconds:', t2-t1


result: 454396537
seconds: 1.29749107361


Too slow. The the array size is just too big at 2e6... Try to vectorize the prime tester instead.

In [8]:
def isprimevec(n):
    """return list of primes below n"""
    ps = []
    v = np.arange(2,n)
    ps.append(v[0]) # start testing with 2
    i = 0
    while len(v)>0:
        t = ps[i]
        vmod = np.mod(v,t)>0 # True for non equally divisible
        v = v[vmod]
        ps.extend(v[v<t**2]) # done testing everything up to n^2
        v = v[v>=t**2]
        i = i+1
    return ps

In [9]:
t1 = time.time() # tic

ps = isprimevec((int(2e6)))
s = np.sum(ps)

t2 = time.time() # toc
print ''
print 'result:', s
print 'seconds:', t2-t1


result: 142913828922
seconds: 0.732959032059


Just barely works fast enough! Only testing up to n^2 is crucial.